# Weight Heatmap Over Time

This notebook generates a heatmap of weight over time from data held in the Health Tracker database and retrieved via the Health Tracker Web Service.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import date, datetime, timedelta

# Reporting date range
start = date(1900, 1, 1)
end = datetime.now() + timedelta(days=1)

# Whether to export the data to a spreadsheet
export_spreadsheet = True

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
# Rolling window size for trend plotting
window = 7

In [ ]:
%run api.ipynb
%run config.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the blood pressure readings
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_weight_measurements(url, token, person_id, start, end)

# Preview the data
df.head()

In [ ]:
# Calmap requires the data frame index to be the date
df.set_index("date", inplace=True)

# Calculate a rolling window and use it to generate trend information
trend_df = df["weight"].rolling(window).mean().diff()
trend_df.name = "Change"

# Remove NaN values
trend_df = trend_df.dropna()

trend_df.head()

In [ ]:
import pandas as pd
from pathlib import Path

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Define the export file name
export_file_name = "Weight-Date-Heatmap"
output_path = export_folder_path / f"{export_file_name}.xlsx"

# Export the data to a spreadsheet
if export_spreadsheet:
    with pd.ExcelWriter(output_path.absolute(), engine="openpyxl") as writer:
        trend_df.to_excel(writer, sheet_name="Trend", index=True)

In [ ]:
import calmap
import matplotlib.pyplot as plt
import matplotlib as mpl

# Create the calendar plot
series = df["weight"]
fig, axes = calmap.calendarplot(
    series,
    cmap="YlOrRd",
    fillcolor="lightgrey",
    linewidth=0.5,
    fig_kws=dict(figsize=(16, 10))
)

# Add a global title
fig.suptitle("Daily Weight Heatmap", fontsize=22, y=0.8)

# Adjust the layout to make space for the colorbar
fig.subplots_adjust(bottom=0.12)

# Create color normalization and colorbar
norm = mpl.colors.Normalize(vmin=series.min(), vmax=series.max())
sm = plt.cm.ScalarMappable(cmap="YlOrRd", norm=norm)
sm.set_array([])

# Add horizontal colorbar below the calendar heatmaps
cbar = fig.colorbar(sm, ax=axes.ravel().tolist(), orientation="horizontal",
                    fraction=0.025, pad=0.08)
cbar.set_label("Weight (Kg)", fontsize=14)

# Export to PNG
if chart_export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}-Daily-Weight.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if chart_export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}-Daily-Weight.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()


In [ ]:
import calmap
import matplotlib.pyplot as plt
import matplotlib as mpl

# Create the calendar plot
fig, axes = calmap.calendarplot(
    trend_df,
    cmap="YlOrRd",
    fillcolor="lightgrey",
    linewidth=0.5,
    fig_kws=dict(figsize=(16, 10))
)

# Add a global title
fig.suptitle("Weight Trend Heatmap", fontsize=22, y=0.8)

# Adjust the layout to make space for the colorbar
fig.subplots_adjust(bottom=0.12)

# Create color normalization and colorbar
norm = mpl.colors.Normalize(vmin=trend_df.min(), vmax=trend_df.max())
sm = plt.cm.ScalarMappable(cmap="YlOrRd", norm=norm)
sm.set_array([])

# Add horizontal colorbar below the calendar heatmaps
cbar = fig.colorbar(sm, ax=axes.ravel().tolist(), orientation="horizontal",
                    fraction=0.025, pad=0.08)
cbar.set_label(f"Rolling {window} Day Change (Kg)", fontsize=14)

# Export to PNG
if chart_export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}-Trend.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if chart_export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}-Trend.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()
